In [ ]:
import os
import torch
from diffusers import StableDiffusion3Pipeline
from PIL import Image

In [ ]:
diffusion_pipe: StableDiffusion3Pipeline = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    torch_dtype=torch.float16,
    token=os.getenv("hf_token"),
)

In [ ]:
image = diffusion_pipe(
    "A cat holding a sign that says hello world",
    negative_prompt="",
    num_inference_steps=28,
    guidance_scale=7.0,
).images[0]
image

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=128,
    lora_alpha=128,
    init_lora_weights="gaussian",
    target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    # lora_dropout=0.1,  # TODO: try this out?
)
diffusion_pipe.transformer.add_adapter(lora_config)

In [ ]:
import torch

num_params = sum(p.numel() for p in diffusion_pipe.transformer.parameters())
print(f"Number of parameters in model: {num_params}")

num_trainable = sum(p.numel() for p in diffusion_pipe.transformer.parameters() if p.requires_grad)
print(f"Number of parameters in model: {num_trainable}")

print(f'Percentage of trainable parameters: {num_trainable / num_params * 100:.2f}%')

In [ ]:
diffusion_pipe.transformer.add_adapter(lora_config)